In [2]:
import pandas as pd
import os

# File paths
titles_path = "../data/raw/titles.csv"
users_path = "../data/raw/users.csv"
sessions_path = "../data/raw/viewing_sessions.csv"

# Check contents of the RAW folder
print("File disponibili in data/raw:")
print(os.listdir("../data/raw"))

# Loading the 3 datasets
titles = pd.read_csv(titles_path)
users = pd.read_csv(users_path)
sessions = pd.read_csv(sessions_path)

print("\nDatasets loaded successfully!\n")

# Shape of the datasets
print("Titles shape:", titles.shape)
print("Users shape:", users.shape)
print("Sessions shape:", sessions.shape)

# First rows
print("\n--- Titles head ---")
display(titles.head(5))

print("\n--- Users head ---")
display(users.head(5))

print("\n--- Sessions head ---")
display(sessions.head(5))

# Columns of the datasets
print("\nColumns in Titles:")
print(titles.columns)

print("\nColumns in Users:")
print(users.columns)

print("\nColumns in Viewing Sessions:")
print(sessions.columns)

# Types of the fields
print("\nDtypes Titles:\n", titles.dtypes)
print("\nDtypes Users:\n", users.dtypes)
print("\nDtypes Sessions:\n", sessions.dtypes)


File disponibili in data/raw:
['._titles.csv', '._users.csv', '._viewing_sessions.csv', 'titles.csv', 'users.csv', 'viewing_sessions.csv']

Datasets loaded successfully!

Titles shape: (2000, 12)
Users shape: (5000, 8)
Sessions shape: (50000, 9)

--- Titles head ---


,title_id,title_name,genre,release_year,origin_country,original_language,duration_min,age_rating,imdb_rating,provider,is_original,date_added
0,1,Title 1,Comedy,1981,DE,German,97,PG,7.6,Netflux,1,2022-08-03
1,2,Title 2,Romance,1985,JP,Japanese,48,G,6.8,Netflux,0,2017-06-14
2,3,Title 3,Thriller,2012,JP,English,90,NC-17,6.7,Streamio,0,2017-06-21
3,4,Title 4,Horror,2017,DE,English,80,R,6.7,CinemaNow,1,2018-02-12
4,5,Title 5,Action,1993,ES,Italian,63,R,8.9,Netflux,0,2019-01-10



--- Users head ---


,user_id,country,signup_date,plan_type,primary_device,age,gender,is_active
0,1,ES,2018-03-29,Premium,Smart TV,59,Other,1
1,2,IT,2021-07-19,Premium,Smart TV,42,Other,1
2,3,UK,2020-03-22,Standard,Mobile,80,Other,1
3,4,AU,2018-03-27,Basic,Mobile,48,Other,0
4,5,IT,2021-09-23,Premium,Smart TV,58,F,1



--- Sessions head ---


,view_id,user_id,title_id,start_time,watch_duration_min,completed,rewatch_flag,device,country
0,1,3159,296,2023-12-17 18:56:00,87,0,0,Smart TV,UK
1,2,1067,1575,2020-09-17 07:01:00,125,0,0,Smart TV,IT
2,3,2222,1830,2023-02-05 15:24:00,70,0,0,Mobile,KR
3,4,1527,847,2020-07-21 06:42:00,150,1,0,Tablet,UK
4,5,2975,1004,2021-05-29 10:34:00,106,0,0,Mobile,UK



Columns in Titles:
Index(['title_id', 'title_name', 'genre', 'release_year', 'origin_country',
       'original_language', 'duration_min', 'age_rating', 'imdb_rating',
       'provider', 'is_original', 'date_added'],
      dtype='object')

Columns in Users:
Index(['user_id', 'country', 'signup_date', 'plan_type', 'primary_device',
       'age', 'gender', 'is_active'],
      dtype='object')

Columns in Viewing Sessions:
Index(['view_id', 'user_id', 'title_id', 'start_time', 'watch_duration_min',
       'completed', 'rewatch_flag', 'device', 'country'],
      dtype='object')

Dtypes Titles:
 title_id               int64
title_name            object
genre                 object
release_year           int64
origin_country        object
original_language     object
duration_min           int64
age_rating            object
imdb_rating          float64
provider              object
is_original            int64
date_added            object
dtype: object

Dtypes Users:
 user_id            int64

In [3]:
# Convertiamo le colonne data/ora nei tipi corretti
titles['date_added'] = pd.to_datetime(titles['date_added'])
users['signup_date'] = pd.to_datetime(users['signup_date'])
sessions['start_time'] = pd.to_datetime(sessions['start_time'])

# Controlliamo velocemente
print(titles['date_added'].head())
print(users['signup_date'].head())
print(sessions['start_time'].head())


0   2022-08-03
1   2017-06-14
2   2017-06-21
3   2018-02-12
4   2019-01-10
Name: date_added, dtype: datetime64[ns]
0   2018-03-29
1   2021-07-19
2   2020-03-22
3   2018-03-27
4   2021-09-23
Name: signup_date, dtype: datetime64[ns]
0   2023-12-17 18:56:00
1   2020-09-17 07:01:00
2   2023-02-05 15:24:00
3   2020-07-21 06:42:00
4   2021-05-29 10:34:00
Name: start_time, dtype: datetime64[ns]


In [4]:
# Add year and month columns to sessions
sessions['year'] = sessions['start_time'].dt.year
sessions['month'] = sessions['start_time'].dt.to_period('M').astype(str)

sessions[['start_time', 'year', 'month']].head()


,start_time,year,month
0,2023-12-17 18:56:00,2023,2023-12
1,2020-09-17 07:01:00,2020,2020-09
2,2023-02-05 15:24:00,2023,2023-02
3,2020-07-21 06:42:00,2020,2020-07
4,2021-05-29 10:34:00,2021,2021-05


In [5]:
# Title aggregation (title_id)
title_metrics = sessions.groupby('title_id').agg(
    total_views=('view_id', 'count'),                  # quante sessioni per quel titolo
    unique_viewers=('user_id', 'nunique'),             # quanti utenti diversi l'hanno visto
    avg_watch_duration=('watch_duration_min', 'mean'), # durata media visione
    completion_rate=('completed', 'mean'),             # % di sessioni completate
    rewatch_rate=('rewatch_flag', 'mean')              # % di sessioni che sono rewatch
).reset_index()

title_metrics.head()


,title_id,total_views,unique_viewers,avg_watch_duration,completion_rate,rewatch_rate
0,1,30,30,99.033333,0.500000,0.266667
1,2,22,22,92.954545,0.409091,0.318182
2,3,19,19,93.736842,0.473684,0.157895
3,4,19,19,106.210526,0.421053,0.105263
4,5,25,25,94.760000,0.560000,0.280000


In [6]:
# Join between titles and title_metrics
catalog = titles.merge(title_metrics, on='title_id', how='left')

# Fill any NaN values (titles never viewed) with 0
catalog[['total_views', 'unique_viewers', 'avg_watch_duration', 
         'completion_rate', 'rewatch_rate']] = catalog[[
    'total_views', 'unique_viewers', 'avg_watch_duration', 
    'completion_rate', 'rewatch_rate'
]].fillna(0)

catalog.head()


,title_id,title_name,genre,release_year,origin_country,original_language,duration_min,age_rating,imdb_rating,provider,is_original,date_added,total_views,unique_viewers,avg_watch_duration,completion_rate,rewatch_rate
0,1,Title 1,Comedy,1981,DE,German,97,PG,7.6,Netflux,1,2022-08-03,30,30,99.033333,0.500000,0.266667
1,2,Title 2,Romance,1985,JP,Japanese,48,G,6.8,Netflux,0,2017-06-14,22,22,92.954545,0.409091,0.318182
2,3,Title 3,Thriller,2012,JP,English,90,NC-17,6.7,Streamio,0,2017-06-21,19,19,93.736842,0.473684,0.157895
3,4,Title 4,Horror,2017,DE,English,80,R,6.7,CinemaNow,1,2018-02-12,19,19,106.210526,0.421053,0.105263
4,5,Title 5,Action,1993,ES,Italian,63,R,8.9,Netflux,0,2019-01-10,25,25,94.760000,0.560000,0.280000


In [7]:
catalog = titles.merge(title_metrics, on='title_id', how='left')

In [8]:
catalog['popularity_score'] = (
    catalog['total_views']*0.4 +
    catalog['unique_viewers']*0.2 +
    catalog['completion_rate']*50 +
    catalog['imdb_rating']*5 +
    catalog['rewatch_rate']*50
)

catalog['engagement_score'] = (
    catalog['avg_watch_duration']/catalog['duration_min'] +
    catalog['completion_rate']*2 +
    catalog['rewatch_rate']
)

catalog['recency_years'] = 2025 - catalog['release_year']
catalog['maturity'] = catalog['age_rating']


In [9]:
catalog[['title_name',
         'popularity_score',
         'engagement_score',
         'recency_years',
         'maturity']].head()

,title_name,popularity_score,engagement_score,recency_years,maturity
0,Title 1,94.333333,2.287629,44,PG
1,Title 2,83.563636,3.072917,40,G
2,Title 3,76.478947,2.146784,13,NC-17
3,Title 4,71.215789,2.275000,8,R
4,Title 5,101.500000,2.904127,32,R


In [10]:

catalog.to_csv("../data/processed/streaming_catalog_enriched.csv", index=False)


In [11]:
output_path = "../data/processed/streaming_catalog_enriched.csv"
catalog.to_csv(output_path, index=False)

output_path


'../data/processed/streaming_catalog_enriched.csv'